## Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -qU pypdf langchain_community
!pip install langchain_chroma langchain_openai --quiet
!pip install chromadb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import random
import numpy as np
import os
import textwrap
import chromadb
import io

from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
import glob
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

from langchain.schema import HumanMessage
from sentence_transformers import SentenceTransformer

## Indexing

In [ ]:
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialise the vector store and embedding functions
chroma_client = chromadb.PersistentClient(path="/path/to/your/folder") # put in your current working directory
embedding_function = SentenceTransformerEmbeddingFunction()
text_collection = chroma_client.get_or_create_collection(name="text_collection", embedding_function=embedding_function)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### set up Claude

In [ ]:
%pip install anthropic --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 6.6 MB/s eta 0:00:00


In [ ]:
import re
import anthropic
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional

In [ ]:
os.environ['ANTHROPIC_API_KEY'] = "your_api_key_here" # put in your own API key here
GLOBAL_CLIENT = anthropic.Anthropic(

    api_key=os.environ.get("ANTHROPIC_API_KEY")
)


### set up the dataset

In [ ]:
# control the ratio of data to be indexed
def manage_dataset(dataset_path, index_ratio):
    random_seed = 42
    random.seed(random_seed)

    file_names = os.listdir(dataset_path)
    random.shuffle(file_names)
    # Calculate split size
    percentage = int(1/(index_ratio/100))
    chunk_size = len(file_names) // percentage  # Calculate base chunk size
    remainder = len(file_names) % percentage  # Calculate any remainder to distribute

    # Create dictionary to store splits
    file_chunks = {}

    start_idx = 0
    for i in range(percentage):
        # Distribute remainder (extra files) to the first few chunks
        end_idx = start_idx + chunk_size + (1 if i < remainder else 0)
        file_chunks[i] = [os.path.join(dataset_path, file_name) for file_name in file_names[start_idx:end_idx]]
        start_idx = end_idx  # Update start index for the next chunk

    return file_chunks

In [ ]:
dataset_path = '/path/to/your/dataset' # put in your local path to the dataset
index_ratio = 0.5
file_chunks = manage_dataset(dataset_path, index_ratio)
print(len(file_chunks[0]))
for file in file_chunks[0]:
    print(file)

### index and save functions

In [ ]:
def clear_vdb():
    global text_collection
    try:
        chroma_client.delete_collection(name="text_collection")
        print("Existing collection deleted.")
    except ValueError:
        print("No existing collection found. Proceeding to create a new one.")

    text_collection = chroma_client.get_or_create_collection(
        name="text_collection",
        embedding_function=embedding_function
    )
    print("New collection created.")

In [ ]:
def load_pdf_dataset(directory_path):
    # Get the paths of all PDF files in the directory.
    pdf_files = glob.glob(f"{directory_path}/*.pdf")
    all_docs = []

    for file_path in pdf_files:
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        all_docs.extend(docs)
    return loader.load()

def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load()

def chunk_text(text, chunk_size = 500, chunk_overlap = 10):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_text(text)


def generate_text_embedding(text):
    return text_model.encode(text).tolist()

### *extract and save locally
This block can be run once to extract features and save. For future runnings this block can be omitted.

In [ ]:
### extract features and save them locally
def extract_and_save_locally_withsplit1(save_path, directory_paths, chunk_size = 500, chunk_overlap = 10):
    ids = []
    uris = []
    embeddings = []
    image_embeddings = []
    text_embeddings = []
    global_id = 0

    documents = []
    metadatas = []


    for file_path in directory_paths:
        print(f"Extracted text for {file_path}")
        documents = load_pdf(file_path)
        pdf_text = " ".join([doc.page_content for doc in documents])
        split_text = chunk_text(pdf_text, chunk_size = chunk_size, chunk_overlap = 10)
        for text in split_text:
            text_embedding = generate_text_embedding(text)
            ids.append(str(global_id))
            uris.append(file_path)
            embeddings.append(text_embedding)

            global_id += 1
            documents.append(text)
            metadatas.append({"image_path": file_path, "text_info": text})


    np.savez(save_path, ids = ids, uris = uris, embeddings = embeddings, documents = documents, metadatas = metadatas)

    return

In [ ]:
index_ratio = 0.5
dataset_path = '/path/to/your/dataset' # put in your local path to the dataset
path1 = '/path/to/save/datasets' # put in the directory where you want to save the extracted features
exp_name = f'vectorstore_{index_ratio}%'
npz_name = '.npz'
save_path = os.path.join(path1, exp_name + npz_name)



split_dictionaries = manage_dataset(dataset_path, index_ratio)
extract_and_save_locally_withsplit1(save_path, split_dictionaries[0], chunk_size = 500, chunk_overlap = 10)

### construct the vectorstore

In [ ]:
# Add embeddings and other info from specified directories to the vector database.
def add_to_vdb3(feature_path, feature_used = 'embeddings'):
    """
    this function is created to make the type of features used in vectorstore a variable
    """
    # documents are not used in later steps, therefore not save to the vectorstore
    # ids = ids, uris = uris, embeddings = embeddings, documents = documents, metadatas = metadatas
    data = np.load(feature_path, allow_pickle=True)
    print(feature_path)
    ids = data['ids'].tolist()
    uris = data['uris'].tolist()
    #documents = data['documents'].tolist()
    metadatas = data['metadatas'].tolist()
    if feature_used == 'embeddings':
        embeddings = data['embeddings']
    elif feature_used == 'image_embeddings':
        embeddings = data['image_embeddings']
    elif feature_used == 'text_embeddings':
        embeddings = data['text_embeddings']
    #embeddings = data['embeddings']
    print(len(ids))
    print(len(uris))
    print(len(embeddings))


    # Add all images to the database at once
    if len(ids)!= 0:
        text_collection.add(ids=ids, uris=uris, embeddings=embeddings, metadatas = metadatas)
        print(f"Added {len(ids)} text embeddings to the database.")

    return

In [ ]:
# construct the vectorstore
exp_no_part = '0.5'
path1 = '/path/to/the/saved/datasets' # put in the path to the saved features
exp_name = f'vectorstore_{exp_no_part}%'
#exp_name = 'vectorstore_20perClass'
npz_name = '.npz'
feature_path = os.path.join(path1, exp_name + npz_name)


# Clear the vector database before adding new images
clear_vdb()

# collect the extracted features from google drive and construct the vectorstore
add_to_vdb3(feature_path)


# Print the total count of images in the vector database
print("text embeddings count in the vectorestore:", text_collection.count())

## Retrieval

In [ ]:
def extract_query_features(query):
    query_embedding = generate_text_embedding(query)

    return query_embedding

In [ ]:
# Retrieve the vector database for similar images.
def query_db1(query, results=3, top_k=500):
    # the top-k here is not useful
    query_features = extract_query_features(query)
    try:
        all_results = text_collection.query(

            query_embeddings=[query_features],
            n_results=text_collection.count(),
            include=['uris', 'distances']
        )
    except Exception as e:
        print(f"Error querying the database: {e}")
        return None

    # Filter results
    filtered_results = filter_results1(all_results, top_k=results)


    retrieved_texts = []
    for file_path in filtered_results['uris'][0]:
        documents = load_pdf(file_path)
        pdf_text = " ".join([doc.page_content for doc in documents])
        retrieved_texts.append(pdf_text)
    while len(retrieved_texts) < 3:
        retrieved_texts.append('')

    return filtered_results, retrieved_texts

In [ ]:
# Filter and sort the retrieved results.
def filter_results1(all_results, top_k):

    # Combined query results
    distances_and_uris_and_ids = list(zip(all_results['distances'][0], all_results['uris'][0], all_results['ids'][0]))


    organised_results = [(dist, uri, id) for dist, uri, id in distances_and_uris_and_ids]

    # Sort results
    sorted_results = sorted(organised_results, key=lambda x: x[0])


    # Select top_k results
    final_results = sorted_results[:top_k]

    # Separate results
    sorted_distances = [dist for dist, _, _ in final_results]
    sorted_uris = [uri for _, uri, _ in final_results]
    sorted_ids = [id for _, _, id in final_results]


    return {
        'ids': [sorted_ids],
        'uris': [sorted_uris],
        'distances': [sorted_distances]
    }

## Generation: Claude for resultsRecord

In [ ]:
# Format the inputs for the prompt template
def format_prompt_inputs1(data, user_query, query):
    # the data here is the retrieved results
    inputs = {}
    inputs['query'] = query
    inputs['user_query'] = user_query


    retrieved_text = ', '.join(data)
    inputs['retrieved_info_texts'] = retrieved_text

    return inputs

In [ ]:
def generate_description_with_rag3(query, user_query):
    try:

        text_fill = '' # not useful in the function, just a placefiller for consistencies of function callings
        results, retrieved_text = query_db1(query, results = 20)
        print(f"Claude Query part is: {query}")
        #print(f"Retrievals: {results['uris']}") # to track which files are retrived
        print(f"No. Retrievals: {len(results['uris'][0])}")


        if results is None or len(results['uris'][0]) == 0:
            print(f"Warning: No results retrieved for {query}")
            return "Unable to generate description due to lack of similar components."

        prompt_input = format_prompt_inputs1(retrieved_text, user_query, query)
        response = claude_vision_chain2(prompt_input)
        print(f"Generations: {response}")
        print("------------------------------------------------------------")
        return response
    except Exception as e:
        print(f"Error in generate_description_with_rag: {str(e)}")
        print("*************************************************************")
        return f"Error generating description: {str(e)}"



def claude_vision_chain2(prompt_input):

    prompt = prompt_input['retrieved_info_texts']
    query = prompt_input['query']
    user_query = prompt_input['user_query']


    message = GLOBAL_CLIENT.messages.create(
    #model="claude-3-5-sonnet-20241022", # this model is deprecated
    model="claude-sonnet-4-20250514",
    max_tokens=1000,
    messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": """There are models and their information."""},
                        {"type": "text", "text": prompt},
                        {"type": "text", "text": """The perspective we want to focus on about these models are:"""},
                        {"type": "text", "text": query},
                        {"type": "text", "text": """Please respond to the request:"""},
                        {"type": "text", "text": user_query},
                        {"type": "text", "text": """Respond with a concise answer in the same phrasing format as in the provided information"""},
                    ]
                }
            ]
    )
    output = message.content[0].text
    return output

### Level 1 queries

In [ ]:
query1 = 'what is the volume, surface area and bounding box volume of model_00219733?'
user_query = """information of model_00219733?
"""
description = generate_description_with_rag3(query1, user_query)

Claude Query part is: what is the volume, surface area and bounding box volume of model_00219733?
No. Retrievals: 20
Generations: The volume of the model_00219733 is 93000.692 cubic MM.
The surface area of the model_00219733 is 20279.800 square MM.
The volume of this model_00219733 bounding box is 185203.611 cubic MM.
------------------------------------------------------------


### Level 2 queries:

In [ ]:
def process_query_resultsRecord(query, user_query, generation_no = 10):
    files = []
    generations = []
    files_no = 0


    # for claude generation:
    description = generate_description_with_rag3(query, user_query)
    # for gpt generation:
    #description = generate_description_with_rag1(query)

    generations.append(description)
    return files, generations

In [ ]:
# volume prompt
query = """the volume of the models
"""
user_query = """list all model, and all models in descending order in terms of their volume
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: the volume of the models

No. Retrievals: 20
Generations: Based on the provided information, here are all the models listed with their volumes in descending order:

1. model_00218789 - 77,855,269.847 cubic MM
2. model_00214261 - 973,788.103 cubic MM
3. model_00217697 - 365,848.342 cubic MM
4. model_00219066 - 121,344.748 cubic MM
5. model_00219090 - 117,363.577 cubic MM (appears multiple times in the data)
6. model_00219733 - 93,000.692 cubic MM
7. model_00219880 - 89,501.364 cubic MM
8. model_00219579 - 21,885.717 cubic MM (appears multiple times in the data)
9. model_00216109 - 16,564.106 cubic MM (appears multiple times in the data)
10. model_00219104 - 5,992.963 cubic MM

All models in descending order by volume:
model_00218789 > model_00214261 > model_00217697 > model_00219066 > model_00219090 > model_00219733 > model_00219880 > model_00219579 > model_00216109 > model_00219104
------------------------------------------------------------


In [ ]:
# hole prompt
query = """total number of round holes of this model
"""
user_query = """list all models in descending order in terms of their number of round holes
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: total number of round holes of this model

No. Retrievals: 20
Generations: Based on the total number of round holes for each model:

model_00214261 has 40 round holes
model_00219090 has 24 round holes  
model_00219579 has 6 round holes
model_00217697 has 3 round holes

The models in descending order by number of round holes:

1. model_00214261 - 40 round holes
2. model_00219090 - 24 round holes
3. model_00219579 - 6 round holes
4. model_00217697 - 3 round holes
------------------------------------------------------------


In [ ]:
# symmetry prompt
query = """symmetry/symmetrical
"""
user_query = """How many different symmetries are there? List examples of models for each type of symmetry
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: symmetry/symmetrical

No. Retrievals: 20
Generations: Based on the provided model information, there is only **1 type of symmetry** represented:

**No Symmetry:**
- All models in the dataset have no symmetry
- Examples include: model_00214261, model_00219579, model_00216109, model_00217697, model_00219090, model_00218789, model_00219733, model_00219104, model_00219880, model_00219066

Every model in the provided dataset explicitly states "This model has no symmetry" and "The axis of symmetry of this model is None."

Therefore, there is **1 symmetry type**: No symmetry, with all provided models serving as examples of this category.
------------------------------------------------------------


### Level 3 queries

In [ ]:
def process_query_resultsRecord(query, user_query, generation_no = 10):
    #results = []
    files = []
    generations = []
    files_no = 0

    # for claude generation:
    description = generate_description_with_rag3(query, user_query)
    # for gpt generation:
    #description = generate_description_with_rag1(query)

    generations.append(description)

    return files, generations

In [ ]:
# similarity based on modelling history
query = """set the direction/origin/scale/extent/operation to/create a line ending at
"""
user_query = """The records of 'set the direction/origin/scale/extent/operation to...' are the modelling history of extrude operations,
and the records of 'createa a line ending at...' are the modelling history of sketch operations.
Based on the modelling history of sketch and extrude operations,
list all models,
and analyse which model is most similar to model_00219066?
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: set the direction/origin/scale/extent/operation to/create a line ending at

No. Retrievals: 20
Generations: Based on the modelling history of sketch and extrude operations, the models listed are:

1. model_00214261
2. model_00216109 
3. model_00219579
4. model_00219733

Analyzing the sketch and extrude operations:

**model_00219066** (reference model):
- Creates circles centered at various coordinates with different radii
- Performs extrude operations with extent one values of 0.025400000000000002 and 0.009000000000000001
- Uses NewBodyFeatureOperation and JoinFeatureOperation
- Creates sketch operations with multiple circles and lines

**Most similar model: model_00216109**

**Analysis:**
- **model_00216109** has the most similar sketch complexity with extensive line and arc creation operations
- Both models use similar extrude extent values (0.025 for model_00216109 vs 0.0254 for model_00219066)
- Both models perform multiple extrude operations with NewBodyFeatu

In [ ]:
# rareness
query = """models
"""
user_query = """To explain the provided info for different models:
The records of 'set the direction/origin/scale/extent/operation to...' are the modelling history of extrude operations,
and the records of 'createa a line ending at...' are the modelling history of sketch operations.
Other records are about geometric features of the models, such as volume and number of holes.
Based on the provided information,
which model seems to be the rarest among all models and why?
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: models

No. Retrievals: 20
Generations: Based on the provided information, **model_00219880** appears to be the rarest among all models.

This model is distinctive because:

- It has only **1 round hole** (with diameter 12.852 MM), making it the model with the fewest holes compared to others that have 9, 24, or 40 holes
- It has the simplest modeling history with only **2 sketches and 1 extrude operation**, while other models have 3-4 sketches and multiple extrude operations
- Its sketching operations consist of only **4 line endings**, creating basic rectangular geometry, whereas other models have complex combinations of circles, arcs, and numerous line segments
- The geometric complexity is minimal compared to models like model_00214261 (40 holes, complex irregular shape) or model_00216109 (9 holes with intricate curved profiles)

The combination of minimal geometric features, simplest construction history, and fewest holes makes model_00219880 the most uncommon

In [ ]:
# manufacturing cost
query = """models
"""
user_query = """To explain the provided info for different models:
The records of 'set the direction/origin/scale/extent/operation to...' are the modelling history of extrude operations,
and the records of 'createa a line ending at...' are the modelling history of sketch operations.
Other records are about geometric features of the models, such as volume and number of holes.
Based on the provided information,
list all models there,
and analyse which model seems to be the most expensive one in terms of manufacturing process and why?
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: models

No. Retrievals: 20
Generations: Based on the provided information, the models listed are:

1. **model_00216109** - A cylindrical part with 9 round holes (diameters: 3.500 MM and 1.500 MM, plus 8.200 MM), volume 16,564.106 cubic MM
2. **model_00214261** - A complex part with 40 round holes (various diameters from 5.924 MM to 95.609 MM), volume 973,788.103 cubic MM  
3. **model_00219880** - A simple rectangular part with 1 round hole (diameter 12.852 MM), volume 89,501.364 cubic MM
4. **model_00219090** - A complex part with 24 round holes (diameters: 8.000 MM and 22.000 MM), volume 117,363.577 cubic MM

**Analysis of manufacturing complexity and cost:**

**model_00214261** appears to be the most expensive one in terms of manufacturing process because:

- **Highest number of holes**: 40 round holes requiring 40 separate drilling/machining operations
- **Largest volume**: 973,788.103 cubic MM requires most raw material
- **Greatest hole diameter variation**: 

In [ ]:
# modelling history generation
query = """models
"""
user_query = """
To explain the provided info for different models:
The records of 'set the direction/origin/scale/extent/operation to...' are the modelling history of extrude operations,
and the records of 'createa a line ending at...' are the modelling history of sketch operations.
Other records are about geometric features of the models, such as volume and number of holes.
Now there is the geometric feature description of an unknown model:
The volume of the model is 93000 cubic MM.
The surface area of the model is 20000 square MM.
The volume of this model bounding box is 180000 cubic MM.
The variance of the Gaussian curvature of this model is 0.041.
The variance of the Mean curvature of this model is 0.000.
The flat plane of this model occupies 1.5%.
The curved surface of this model occupies 98.5%.
The straight edge of this model occupies 58%.
The curved edge of this model occupies 42%.
This model has no symmetry.
The axis of symmetry of this model is None.
Total number of round holes of this model is 0.
suggest the possible modelling history of this model
"""
files, generations = process_query_resultsRecord(query, user_query)

Claude Query part is: models

No. Retrievals: 20
Generations: Based on the geometric features of the unknown model, I can suggest a possible modeling history:

**Model Analysis:**
- Volume: 93,000 cubic MM (moderate size)
- High curved surface percentage (98.5%) with minimal flat surfaces (1.5%)
- No round holes
- No symmetry
- Moderate Gaussian curvature variance (0.041)

**Suggested Modeling History:**

The model creates a sketch named Sketch 1 with the following features:
The model creates a line ending at (0.0, 0.0).
The model creates a line ending at (0.045, 0.0).
The model creates a line ending at (0.045, 0.035).
The model creates an arc ending at (0.035, 0.045), sweeping 90 degrees clockwise.
The model creates a line ending at (0.01, 0.045).
The model creates an arc ending at (0.0, 0.035), sweeping 90 degrees clockwise.
The model creates a line ending at (0.0, 0.0).

The model performs an extrude operation named Extrude 1 with the following parameters:
The model sets the directi